# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [24]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

# abrir o navegador
nav = webdriver.Chrome()

# Passo 1: Pegar cotação do Dólar 

# entrar no google
nav.get("https://www.google.com.br/")
# pesquisar no google por "cotação dólar"
nav.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dólar")
nav.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
# pegar a cotação que ta no google
cotacao_dolar = nav.find_element(By.XPATH,
'//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")

print(cotacao_dolar)


# Passo 2: Pegar cotação do Euro 

# entrar no google
nav.get("https://www.google.com.br/")
nav.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro")
nav.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

# pegar a cotação que ta no google
cotacao_euro = nav.find_element(By.XPATH,
'//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")

print(cotacao_euro)

# Passo 3: Pegar cotação do Ouro 

nav.get('https://www.melhorcambio.com/ouro-hoje')
cotacao_ouro = nav.find_element(By.XPATH, '//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(',', '.')
print(cotacao_ouro)

nav.quit()

5.270099999999999
5.442695775000001
296.27


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [26]:
# Passo 4: Atualizar a base de preços (atualizar o preço de compra e o de venda)
import pandas as pd

tabela = pd.read_excel("Produtos.xlsx")
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [29]:
# atualuzar a caluna de cotação
tabela.loc[tabela["Moeda"] == "Dólar", "Cotação"] = float(cotacao_dolar)
tabela.loc[tabela["Moeda"] == "Euro", "Cotação"] = float(cotacao_euro)
tabela.loc[tabela["Moeda"] == "Ouro", "Cotação"] = float(cotacao_ouro)

# atualuzar a caluna de preço de compra = preço original * cotação 
tabela['Preço de Compra'] = tabela['Preço Original'] * tabela['Cotação']

# atualuzar a caluna preço de venda = preço de compra * margem
tabela['Preço de Venda'] = tabela['Preço de Compra'] * tabela['Margem']

display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.270100,5270.047299,1.40,7378.066219
1,Carro Renault,4500.00,Euro,5.442696,24492.130988,2.00,48984.261975
2,Notebook Dell,899.99,Dólar,5.270100,4743.037299,1.70,8063.163408
3,IPhone,799.00,Dólar,5.270100,4210.809900,1.70,7158.376830
4,Carro Fiat,3000.00,Euro,5.442696,16328.087325,1.90,31023.365918
5,Celular Xiaomi,480.48,Dólar,5.270100,2532.177648,2.00,5064.355296
6,Joia 20g,20.00,Ouro,296.270000,5925.400000,1.15,6814.210000


### Agora vamos exportar a nova base de preços atualizada

In [30]:
# Passo 5: Exportar a base de preços atualizada 
tabela.to_excel("Produtos novo.xlsx", index=True)

